# Dihedral parameters for ethane and ethene

Thanks to the simple functional form of molecular-mechanics force fields, their parameters can be fitted one by one or in small groups as a first approximation, by focusing on a few degrees of freedom. In this notebook we are going to fit and analyze the dihedral parameters for ethane (C<sub>2</sub>H<sub>6</sub>), ethene/ethylene (C<sub>2</sub>H<sub>4</sub>), to see how the nature of the C-C bond affects the rotational freedom of the molecule. As in the case of the dimer, I have used NWChem to sample the energy at different values of the coordinate of interest. In this case, we focus on a dihedral angle relative to the plane defined by the two carbon atoms and one of the hydrogens:

![Reference plane](reference.png "Reference plane")

For ethane, $\Phi$ is the relative angle between the two CH<sub>3</sub> tetrahedra:

![Phi in ethane](ethane.png "Phi in ethane")

For ethene, it is the angle between the planes defined by the two CH<sub>2</sub> groups:

![Phi in ethene](ethene.png "Phi in ethene")

In [ ]:
%matplotlib widget


from IPython.display import Markdown
from IPython.display import IFrame

import string

import numpy as np
import scipy as sp
import scipy.misc
import scipy.interpolate
import scipy.optimize
import matplotlib
import matplotlib.pyplot as plt

import ase
import ase.units

alphabet = string.ascii_lowercase
colors = matplotlib.rcParams["axes.prop_cycle"].by_key()["color"]
matplotlib.rcParams["figure.figsize"] = (8, 8)

In [ ]:
# Load and plot the data from NWCHem's output
data_ane = np.loadtxt("ethane_e_vs_phi.txt")
phi_ane = data_ane[:, 0]
E_ane = data_ane[:, 1] * ase.units.Hartree
E_ane -= E_ane[-1]

data_ene = np.loadtxt("ethene_e_vs_phi.txt")
phi_ene = data_ene[:, 0]
E_ene = data_ene[:, 1] * ase.units.Hartree
E_ene -= E_ene[-1]

plt.figure()
plt.polar(phi_ane * np.pi / 180., E_ane, label=r"$E_{\mathrm{pot}}$", color=colors[0])
ax = plt.gca()
ax.set_theta_zero_location("N")
plt.title("Ethane")
plt.legend(loc="best")
plt.show()

plt.figure()
plt.polar(phi_ene * np.pi / 180., E_ene, label=r"$E_{\mathrm{pot}}$", color=colors[1])
ax = plt.gca()
ax.set_theta_zero_location("N")
plt.title("Ethene")
plt.legend(loc="best")
plt.show()


# Fitting the results to the Ryckaert-Bellemans form

We will fit each of these curves to a function of the form:

$$E_{\mathrm{pot}}=V_0 + \frac{V_1}{2}\left[1+\cos\Phi\right]+ \frac{V_2}{2}\left[1-\cos\left(2\Phi\right)\right]+\frac{V_3}{2}\left[1+\cos\left(3\Phi\right)\right]+ \frac{V_4}{2}\left[1-\cos\left(4\Phi\right)\right]$$

There is a new $V_0$ term absent from the slides, since we need to accommodate the arbitrary origin of energies chosen by the DFT program.

It is interesting to see how each term of this expression looks like in a polar plot:

In [ ]:
plt.figure()

phi = np.linspace(0., 2. * np.pi, num=1000)

for i in range(1, 5):
    parity = (i - 1) % 2
    plt.polar(phi, 1. + (1., -1.)[parity] * np.cos(i * phi),
              label=r"$1 {} \cos\left({}\phi\right)$".format(("+", "-")[parity], i if i > 1 else ""),
              color=colors[i], lw=2.)
ax = plt.gca()
ax.set_theta_zero_location("N")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
def ryckaert_bellemans(phi, V0, V1, V2, V3, V4):
    """Implementation of the Ryckaert-Bellemans dihedral term."""
    return V0 + .5 * (V1 * (1. + np.cos(phi)) + V2 * (1. - np.cos(2. * phi)) +
                      V3 * (1. + np.cos(3. * phi)) + V4 * (1. - np.cos(4. * phi)))

## Fit for ethane

In [ ]:
result = scipy.optimize.curve_fit(ryckaert_bellemans, phi_ane * np.pi / 180., E_ane, p0=(E_ane.mean(), 0., 0., 0., 0.))
V0, V1, V2, V3, V4 = result[0]

display(Markdown(r"""
Ryckaert-Bellemans for ethane:

- $V_0 = {:.4f}\;\mathrm{{eV}}$
- $V_1 = {:.4f}\;\mathrm{{eV}}$
- $V_2 = {:.4f}\;\mathrm{{eV}}$
- $V_3 = {:.4f}\;\mathrm{{eV}}$
- $V_4 = {:.4f}\;\mathrm{{eV}}$
""".format(V0, V1, V2, V3, V4)))

plt.figure()
plt.polar(phi_ane * np.pi / 180., E_ane, label="DFT", color=colors[0])
plt.polar(phi, ryckaert_bellemans(phi, V0, V1, V2, V3, V4), label="Ryckaert-Bellemans", color=colors[1])
plt.title("Ethane")
plt.legend(loc="best")
ax = plt.gca()
ax.set_theta_zero_location("N")
plt.show()

## Fit for ethene

In [ ]:
result = scipy.optimize.curve_fit(ryckaert_bellemans, phi_ene * np.pi / 180., E_ene, p0=(E_ene.mean(), 0., 0., 0., 0.))
V0, V1, V2, V3, V4 = result[0]

display(Markdown(r"""
Ryckaert-Bellemans for ethene:

- $V_0 = {:.4f}\;\mathrm{{eV}}$
- $V_1 = {:.4f}\;\mathrm{{eV}}$
- $V_2 = {:.4f}\;\mathrm{{eV}}$
- $V_3 = {:.4f}\;\mathrm{{eV}}$
- $V_4 = {:.4f}\;\mathrm{{eV}}$
""".format(V0, V1, V2, V3, V4)))

plt.figure()
plt.polar(phi_ene * np.pi / 180., E_ene, label="DFT", color=colors[0])
plt.polar(phi, ryckaert_bellemans(phi, V0, V1, V2, V3, V4), label="Ryckaert-Bellemans", color=colors[1])
plt.title("Ethene")
plt.legend(loc="best")
ax = plt.gca()
ax.set_theta_zero_location("N")
plt.show()